In [1]:
%matplotlib widget

from pathlib import Path
from typing import Tuple

import ipywidgets as widgets
import numpy as np
import plotly.graph_objects as go
from IPython.display import clear_output
from ipywidgets import Layout
from ipywidgets import interactive

from viv1t import data
from viv1t.utils import utils

TICK_FONTSIZE = 10
LABEL_FONTSIZE = 11
TITLE_FONTSIZE = 12

FPS = 30

DATA_DIR = Path("../data/sensorium")
MOUSE_IDS = ["B", "C", "E"]
NEURON_COORDINATES = {
    mouse_id: data.get_neuron_coordinates(mouse_id=mouse_id, to_tensor=False)
    for mouse_id in MOUSE_IDS
}
TUNING = {
    mouse_id: utils.load_tuning(data.METADATA_DIR, mouse_id=mouse_id)
    for mouse_id in MOUSE_IDS
}
DIRECTIONS = np.array([0, 45, 90, 135, 180, 225, 270, 315], dtype=np.int32)


ORIENTATION_COLORMAPS = {
    0: "#1d4e31",  # 0
    1: "#EA0B13",  # 45
    2: "#FEBC16",  # 90
    3: "#1D4AA3",  # 135
}
DIRECTION_COLORMAPS = {
    0: "#1d4e31",  # 0
    1: "#EA0B13",  # 45
    2: "#FEBC16",  # 90
    3: "#1D4AA3",  # 135
    4: "#2ea45d",  # 180
    5: "#ff686e",  # 225
    6: "#ffdf90",  # 270
    7: "#718dc5",  # 315
}

get_ticks = lambda coors: np.linspace(
    np.floor(np.min(coors) * 0.1) * 10 - 20,
    np.ceil(np.max(coors) * 0.1) * 10 + 20,
    5,
    dtype=int,
)
X_TICKS = {
    mouse_id: get_ticks(NEURON_COORDINATES[mouse_id][:, 0]) for mouse_id in MOUSE_IDS
}
Y_TICKS = {
    mouse_id: get_ticks(NEURON_COORDINATES[mouse_id][:, 1]) for mouse_id in MOUSE_IDS
}
Z_TICKS = {
    mouse_id: get_ticks(NEURON_COORDINATES[mouse_id][:, 2]) for mouse_id in MOUSE_IDS
}

In [2]:
def get_neurons_within_range(
    neuron_coordinates: np.ndarray,
    x_range: Tuple[int, int],
    y_range: Tuple[int, int],
    z_range: Tuple[int, int],
):
    """Return neurons that are within the x, y and z ranges"""
    x_neurons = np.where(
        np.logical_and(
            neuron_coordinates[:, 0] >= x_range[0],
            neuron_coordinates[:, 0] <= x_range[1],
        )
    )[0]
    y_neurons = np.where(
        np.logical_and(
            neuron_coordinates[:, 1] >= y_range[0],
            neuron_coordinates[:, 1] <= y_range[1],
        )
    )[0]
    z_neurons = np.where(
        np.logical_and(
            neuron_coordinates[:, 2] >= z_range[0],
            neuron_coordinates[:, 2] <= z_range[1],
        )
    )[0]
    neurons = np.intersect1d(x_neurons, y_neurons, assume_unique=True)
    neurons = np.intersect1d(neurons, z_neurons, assume_unique=True)
    return neurons

In [3]:
clear_output(wait=True)

margin = "0.1em 1em 0.1em"
style = {"description_width": "initial", "indent": False}
layout = Layout(width="20em", margin=margin)

mouse_id_dropdown = widgets.Dropdown(
    options=MOUSE_IDS,
    value="B",
    description="Mouse ID",
    style=style,
    layout=layout,
)

si_dropdown = widgets.Dropdown(
    options=["OSI", "DSI"],
    value="OSI",
    description="SI type",
    style=style,
    layout=layout,
)

si_threshold_slider = widgets.FloatSlider(
    min=0.01,
    max=1.0,
    step=0.01,
    value=0.3,
    description="SI threshold",
    disabled=False,
    style=style,
    layout=layout,
)

opacity_slider = widgets.FloatSlider(
    min=0.0,
    max=1.0,
    step=0.01,
    value=0.8,
    description="Marker opacity",
    style=style,
    layout=layout,
)

marker_size_slider = widgets.IntSlider(
    min=1,
    max=10,
    step=1,
    value=5,
    description="Marker size",
    style=style,
    layout=layout,
)

reset_button = widgets.Button(description="Reset", style=style, layout=layout)


coordinate_layout = Layout(width="25em", margin=margin)

x_coordinates_slider = widgets.IntRangeSlider(
    value=[0, 10],
    step=10,
    description="x range (μm)",
    disabled=False,
    style=style,
    layout=coordinate_layout,
)

y_coordinates_slider = widgets.IntRangeSlider(
    value=[0, 10],
    step=10,
    description="y range (μm)",
    disabled=False,
    style=style,
    layout=coordinate_layout,
)

z_coordinates_slider = widgets.IntRangeSlider(
    value=[0, 10],
    step=10,
    description="z range (μm)",
    disabled=False,
    style=style,
    layout=coordinate_layout,
)


def set_slider(slider, min_value, max_value):
    if slider.max < min_value:
        slider.max = max_value
        slider.min = min_value
    else:
        slider.min = min_value
        slider.max = max_value


def set_mouse_id(mouse_id: str):
    si_type = si_dropdown.value
    si_values = TUNING[mouse_id][si_type]
    si_min, si_max = np.nanmin(si_values), np.nanmax(si_values)
    print(si_type, si_values)

    set_slider(si_threshold_slider, si_min, si_max)
    si_threshold_slider.disabled = False
    si_threshold_slider.value = 0.3

    x_min = NEURON_COORDINATES[mouse_id][:, 0].min()
    x_max = NEURON_COORDINATES[mouse_id][:, 0].max()
    set_slider(x_coordinates_slider, x_min, x_max)
    x_coordinates_slider.value = [x_coordinates_slider.min, x_coordinates_slider.max]

    y_min = NEURON_COORDINATES[mouse_id][:, 1].min()
    y_max = NEURON_COORDINATES[mouse_id][:, 1].max()
    set_slider(y_coordinates_slider, y_min, y_max)
    y_coordinates_slider.value = [y_coordinates_slider.min, y_coordinates_slider.max]

    z_max = NEURON_COORDINATES[mouse_id][:, 2].max()
    z_min = NEURON_COORDINATES[mouse_id][:, 2].min()
    set_slider(z_coordinates_slider, z_min, z_max)
    z_coordinates_slider.value = [z_coordinates_slider.min, z_coordinates_slider.max]


figure = go.FigureWidget()


def reset(b):
    si_threshold_slider.value = 0.3
    opacity_slider.value = 0.8
    marker_size_slider.value = 5
    x_coordinates_slider.value = [x_coordinates_slider.min, x_coordinates_slider.max]
    y_coordinates_slider.value = [y_coordinates_slider.min, y_coordinates_slider.max]
    z_coordinates_slider.value = [z_coordinates_slider.min, z_coordinates_slider.max]
    figure.data = []
    figure.layout = {}
    animate_sample(
        mouse_id=mouse_id_dropdown.value,
        si=si_dropdown.value,
        si_threshold=si_threshold_slider.value,
        opacity=opacity_slider.value,
        marker_size=marker_size_slider.value,
        x_range=x_coordinates_slider.value,
        y_range=y_coordinates_slider.value,
        z_range=z_coordinates_slider.value,
    )


def animate_sample(
    mouse_id: str,
    si: str,
    si_threshold: float,
    opacity: float,
    marker_size: int,
    x_range: Tuple[int, int],
    y_range: Tuple[int, int],
    z_range: Tuple[int, int],
):
    neuron_coordinates = NEURON_COORDINATES[mouse_id]

    si_values = TUNING[mouse_id][si]
    si_neurons = np.where(si_values >= si_threshold)[0]

    directions = DIRECTIONS
    tuning_curves = TUNING[mouse_id]["tuning_curve"]

    colormaps = DIRECTION_COLORMAPS
    if si == "OSI":
        # combine opposite directions for a single orientation value
        directions = directions[:4]
        tuning_curves = (tuning_curves[:, :4] + tuning_curves[:, 4:]) / 2
        colormaps = ORIENTATION_COLORMAPS
    preferred_directions = np.argmax(tuning_curves, axis=-1)

    if not figure.data:
        for i, direction in enumerate(directions):
            figure.add_trace(
                go.Scatter3d(
                    name=f"{direction}°",
                    mode="markers",
                    marker={"color": colormaps[i]},
                )
            )
        # add a dummy trace to keep depth (z-axis) consistent
        depths = np.unique(neuron_coordinates[:, 2])
        figure.add_trace(
            go.Scatter3d(
                mode="markers",
                x=[neuron_coordinates[0, 0]] * len(depths),
                y=[neuron_coordinates[0, 1]] * len(depths),
                z=depths,
                marker={"size": 1, "opacity": 0},
                showlegend=False,
            )
        )
        # change default camera angle
        figure.update_layout(scene_camera=dict(center=dict(x=0, y=0, z=-0.2)))

    range_neuron = get_neurons_within_range(
        neuron_coordinates, x_range=x_range, y_range=y_range, z_range=z_range
    )

    with figure.batch_update():
        for i in range(len(directions)):
            neurons = np.where(preferred_directions == i)[0]
            neurons = np.intersect1d(neurons, si_neurons, assume_unique=True)
            neurons = np.intersect1d(neurons, range_neuron, assume_unique=True)
            figure.data[i].x = neuron_coordinates[neurons, 0]
            figure.data[i].y = neuron_coordinates[neurons, 1]
            figure.data[i].z = neuron_coordinates[neurons, 2]
            figure.data[i].marker.size = marker_size
            figure.data[i].marker.opacity = opacity

    layout = go.Layout(
        margin={"l": 0, "r": 0, "b": 0, "t": 0},
        height=850,
        width=1400,
        scene={
            "aspectmode": "manual",
            "aspectratio": {"x": 1, "y": 1, "z": 0.6},
            "xaxis": {
                "title": "x coordinate (μm)",
                "nticks": 5,
                "range": [X_TICKS[mouse_id][0], X_TICKS[mouse_id][-1]],
            },
            "yaxis": {
                "title": "y coordinate (μm)",
                "nticks": 5,
                "range": [Y_TICKS[mouse_id][0], Y_TICKS[mouse_id][-1]],
            },
            "zaxis": {
                "title": "z coordinate (μm)",
                "nticks": 5,
                "range": [Z_TICKS[mouse_id][0], Z_TICKS[mouse_id][-1]],
            },
        },
        legend={
            "itemsizing": "constant",
            "xanchor": "auto",
            "x": 0,
            "yanchor": "auto",
            "y": 1,
            "font": {"size": 16},
        },
    )

    figure.update_scenes(zaxis_autorange="reversed")
    figure.update_layout(layout)


interactive(set_mouse_id, mouse_id=mouse_id_dropdown)
interactive(
    animate_sample,
    mouse_id=mouse_id_dropdown,
    si=si_dropdown,
    si_threshold=si_threshold_slider,
    opacity=opacity_slider,
    marker_size=marker_size_slider,
    x_range=x_coordinates_slider,
    y_range=y_coordinates_slider,
    z_range=z_coordinates_slider,
)
reset_button.on_click(reset)

display(
    widgets.HBox([mouse_id_dropdown, si_dropdown, si_threshold_slider, reset_button])
)
display(widgets.HBox([opacity_slider, marker_size_slider]))
display(
    widgets.HBox([x_coordinates_slider, y_coordinates_slider, z_coordinates_slider])
)
display(figure)

FigureWidget({
    'data': [{'marker': {'color': '#1d4e31', 'opacity': 0.8, 'size': 5},
              'mode': 'markers',
              'name': '0°',
              'type': 'scatter3d',
              'uid': 'd2e330e0-374e-4e9f-8ea0-58d9bdf76833',
              'x': array([-736., -932., -686., ..., -481., -639., -942.],
                         shape=(488,), dtype=float32),
              'y': array([-584., -472., -445., ..., -487., -374., -464.],
                         shape=(488,), dtype=float32),
              'z': array([325., 325., 325., ..., 425., 425., 425.], shape=(488,), dtype=float32)},
             {'marker': {'color': '#EA0B13', 'opacity': 0.8, 'size': 5},
              'mode': 'markers',
              'name': '45°',
              'type': 'scatter3d',
              'uid': '3b1a95ee-dbe3-408b-95be-12028f34a901',
              'x': array([-759., -497., -863., ..., -796., -746., -814.],
                         shape=(618,), dtype=float32),
              'y': array([-606., -588.